In [ ]:
import zipfile
with zipfile.ZipFile('./data/ReutersNews106521.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/2006-2013')

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import pickle
from striprtf.striprtf import rtf_to_text
import string
from os import listdir
from datetime import datetime
from tqdm import tqdm
import cathay.re.SVO_final as SVO
import re
from cathay.config import ApplicationConfig
import boto3
from multiprocessing import Pool
import torch.multiprocessing as mp
import nltk
from transformers import *
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch import optim
import torch.nn.functional as F

# 2006-2013

In [123]:
def Preprocess(dataset):
    datas = []
    for folder in tqdm(dataset):
        files = listdir(path + folder)
        for file in files:
            if file[0] != '.':
                with open(path + folder + '/' + file, 'r') as f:
                    new = f.readlines()

                if new != []:
                    data = {}
                    new = [x.replace('\n', '') for x in new if x[:2] != '\n' and len(x) > 3]
                    data['title'] = new[0].replace('-- ', '')
                    data['source'] = 'Reuters News'
                    date = new[2].replace('-- ', '')
                    data['date'] = datetime.strptime(' '.join(date.split(' ')[:4]), '%a %b %d, %Y').strftime('%Y%m%d')
                    data['text'] = ''.join(new[4:])
                    datas.append(data)
                
    return datas

In [124]:
path = './financial-news-dataset-master/ReutersNews106521/'
folders = [x for x in listdir(path) if len(x) == 8]

n_workers = 5
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(folders) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(folders)
        else:
            batch_end = (len(folders) // n_workers) * (i + 1)

        batch = folders[batch_start: batch_end]
        results[i] = pool.apply_async(Preprocess, [batch])

    pool.close()
    pool.join()

processed = []
for result in results:
    processed += result.get()

100%|██████████| 365/365 [20:46<00:00,  3.42s/it]


In [125]:
len(processed)

69944

In [126]:
path = './data/2006-2013/'
folders = [x for x in listdir(path) if len(x) == 8]

n_workers = 5
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(folders) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(folders)
        else:
            batch_end = (len(folders) // n_workers) * (i + 1)

        batch = folders[batch_start: batch_end]
        results[i] = pool.apply_async(Preprocess, [batch])

    pool.close()
    pool.join()

for result in results:
    processed += result.get()

100%|██████████| 152/152 [15:52<00:00,  6.26s/it]


In [127]:
len(processed)

106494

In [133]:
news = pd.DataFrame({'id':range(len(processed)), 'title':[x['title'] for x in processed], 'date':[x['date'] for x in processed], 'source':[x['source'] for x in processed], 'document_body':[x['text'] for x in processed]})

In [135]:
with open('2006-2013.pkl', 'wb') as f:
    pickle.dump(news, f)

# RTF to DataFrame (2014-2019)

In [3]:
years = ['2014', '2015', '2016', '2017', '2018', '2019']

In [4]:
all_news = []
for year in years:
    mypath = './data/' + year + '/'
    files = listdir(mypath)
    for file in tqdm(files):
        with open(mypath + file, 'r') as f:
            data = f.readlines()
            
        for i in data:
            if i[:4] == '\\par' and rtf_to_text(i) != '\n':
                new = {}
                text = rtf_to_text(i)
                text = text.split('\n')
                text = [x for x in text if x != '']
                if len(text) < 5:
                    continue
                    
                count = 0
                # 總字數
                while '字' not in text[count]:
                    count += 1
                # 作者
                if 'By' in text[count-1] and count-2 >= 0:
                    new['title'] = text[count-2]
                else:
                    new['title'] = text[count-1]
                new['date'] = ''.join([text[count+1].split(' ')[x].zfill(2) for x in [0, 2, 4]])
                new['source'] = text[count+2]
                count = count + 2
                
                while '(c)' not in text[count] and 'Copyright' not in text[count]:
                    count += 1
                if text[count+1][-1] in string.ascii_letters:
                    s = text[count+1] + '.'
                else:
                    s = text[count+1]
                for j in range(count+2, len(text)-2):
                    if text[j][-1] in string.ascii_letters:
                        s += ' ' + text[j] + '.'
                    else:
                        s += ' ' + text[j]
                new['text'] = s
                all_news.append(new)
news = pd.DataFrame({'id':range(len(all_news)), 'title':[x['title'] for x in all_news], 'date':[x['date'] for x in all_news], 'source':[x['source'] for x in all_news], 'document_body':[x['text'] for x in all_news]})

100%|██████████| 97/97 [01:18<00:00,  1.23it/s]


In [148]:
with open('2014-2019.pkl', 'wb') as f:
    pickle.dump(news, f)

# Event Extraction

In [2]:
# with open('2014-2019.pkl', 'rb') as f:
#     news = pickle.load(f)
with open('2006-2013.pkl', 'rb') as f:
    news = pickle.load(f)

In [3]:
news.head()

,id,title,date,source,document_body
0,0,Exxon Mobil offers plan to end Alaska dispute,20061020,Reuters News,"ANCHORAGE, Alaska (Reuters) - Exxon Mobil ( ..."
1,1,"Hey buddy, can you spare $600 for a Google share?",20061020,Reuters News,SAN FRANCISCO/NEW YORK (Reuters) - Wall Stre...
2,2,AOL CEO says sales may shrink for two years -p...,20061021,Reuters News,FRANKFURT (Reuters) - Internet service provi...
3,3,"Fed to keep hawkish tone, hold rates steady",20061022,Reuters News,WASHINGTON (Reuters) - The central bank is e...
4,4,Pluspetrol says losing $2.4 mln/day in Peru pr...,20061021,Reuters News,"LIMA, Peru (Reuters) - Argentine oil company..."


In [4]:
news_dict = news.to_dict('records')

### Title

In [5]:
aws_nlu_config =  ApplicationConfig.get_aws_nlu_config()
comprehend = boto3.client(aws_access_key_id=aws_nlu_config['access_key'], aws_secret_access_key=aws_nlu_config['secret_key'], service_name='comprehend', region_name=aws_nlu_config['region'])

In [6]:
def title_preprocess(sent, comprehend):
    if sent.find('-') == 8 and sent[:6] == 'UPDATE':
        sent = sent[9:]
    if sent.find('-') == 8 and sent[:6] == 'WRAPUP':
        sent = sent[9:]
    if sent.find('-') == 3 and sent[:3] == 'RPT':
        sent = sent[5:]
    while sent.find('-') != -1 and sent[:sent.find('-')].isupper():
        sent = sent[sent.find('-')+1:]
    sent = sent.replace(' - ',' ')
    sent = sent.replace("''",' ')
    sent = re.sub("[+\!\/\\_$%^*()+.:\"“”]+|[+——！，。？、~@#￥%……&*（）：`]+", '', sent)
    sent = sent.replace('\\',' ')
    sent = sent.replace('  ',' ')
    if sent[0] in ['-', ' ']:
        sent = sent[1:]
        
    # 只留句首以及專有名詞大寫
    idx = sent.find(' ')
    # 找專有名詞
    entity = comprehend.detect_entities(Text=sent[idx:], LanguageCode='en')['Entities']
    b = sent[idx:].lower()
    b = b.lower()
    s = ''
    end = 0
    for i in entity:
        s += b[end:i['BeginOffset']]
        s += i['Text']
        end = i['EndOffset']
    s += b[end:]
    sent = sent[:idx] + s
    return sent

In [7]:
# 使用Title，過濾掉 'The Year Ahead' 和 'Quick Takes'
def Event_extrations(dataset):
    processed = []
    for sample in tqdm(dataset):
        sent = re.split(':|;|---', sample['title'])[-1]
        if sent != '' and 'The Year Ahead' not in sent and 'Quick Takes' not in sent:
            tmp = {}
            try:
                sent = title_preprocess(sent, comprehend)
                svo = SVO.SVO(sent)
                svo_result = svo.find_svo()
                tmp['id'] = sample['id']
                tmp['date'] = sample['date']
                tmp['title'] = sample['title']
                tmp['title_SVO'] = svo_result
                processed.append(tmp)
            except:
                print(sent)
    return processed

In [8]:
n_workers = 8
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(news_dict) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(news_dict)
        else:
            batch_end = (len(news_dict) // n_workers) * (i + 1)

        batch = news_dict[batch_start: batch_end]
        results[i] = pool.apply_async(Event_extrations, [batch])

    pool.close()
    pool.join()

processed = []
for result in results:
    processed += result.get()

  3%|▎         | 442/13311 [04:45<2:03:09,  1.74it/s]

Some 4 trillion wiped off world stocks in 2 weeks


 18%|█▊        | 2434/13311 [26:11<2:24:25,  1.26it/s]

SocGen boss survives and says bank can too


 26%|██▋       | 3527/13311 [38:07<1:33:21,  1.75it/s]

...

 67%|██████▋   | 8944/13311 [1:29:47<29:02,  2.51it/s]  

Obama on attack in foreign policy debate, but Romney steady


 97%|█████████▋| 12971/13311 [2:00:06<02:30,  2.25it/s]

US Airways fined 12 million over disabilities infractions


100%|██████████| 13317/13317 [2:05:58<00:00,  1.76it/s]


In [19]:
# with open('news_preprocessed.pkl', 'wb') as f:
#     pickle.dump(processed, f)

### Integrate to SVO

In [21]:
with open('news_preprocessed.pkl', 'rb') as f:
    processed = pickle.load(f)

In [23]:
# 標題的整合
class to_SVO():
    def __init__(self):
        self._be = ['is', 'are', 'am', 'was', 'were']
        
    def to_SVO(self, data):
        if data == 'Sentence can not find SVO.':
            return []
        
        self._data_key = data.keys()
        self._results = []
        # key: main / which..., value: [{}, {}]->dictionary(keys['subject', 'predicate', 'object']
        for key, value in data.items(): 
            for svos in value:                
                # 只有主詞
                if svos['subject'] != [] and svos['predicate'] == [] and svos['object'] == []:
                    self._Subject_only(svos)
                    
                # 沒有受詞                
                if svos['subject'] != [] and svos['predicate'] != [] and svos['object'] == []:
                    self._No_Object(svos)
                
                # 主動受詞都有
                if svos['subject'] != [] and svos['predicate'] != [] and svos['object'] != []:
                    self._Complete(svos)
        
        return self._results
    
    def _Attr_flatten(self, attrs):
        attr_flatten = []
        for attr in [x for x in attrs if isinstance(x, dict) == False and x != None]:
            attr_flatten.append(attr)
        for attr in [x for x in attrs if isinstance(x, dict) == True]:
            for i in ['predicate', 'object']:
                for j in attr[i]:
                    attr_flatten.append(j[0])
                    attr_flatten += self._Attr_flatten(j[1])
        return attr_flatten
    
    def _Subject_only(self, svos):
        # svo: ('', [])
        for svo in svos['subject']:
            # 主詞非dic的Attr
            S_attr = []
            for attr in [x for x in svo[1] if isinstance(x, dict) == False and x != None]:
                S_attr.append(attr)
                
            # 主詞Attr含有動詞，可形成事件
            if True in [isinstance(x, dict) for x in svo[1]]:
                for attr in [x for x in svo[1] if isinstance(x, dict) == True]:
                    if attr['object'] != []:
                        self._results.append([(svo[0], ' '.join(S_attr)), 
                                        (attr['predicate'][0][0], ' '.join(self._Attr_flatten(attr['predicate'][0][1]))), 
                                        (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                    
                    # dictionary沒有object
                    else:
                        self._results.append([(svo[0], ' '.join(S_attr)), 
                                        (attr['predicate'][0][0], ' '.join(self._Attr_flatten(attr['predicate'][0][1])))])
                        
            # 主詞Attr沒有動詞，無法形成事件
            else:
                self._results.append([(svo[0], ' '.join(S_attr))])
    
    def _No_Object(self, svos):
        # 連接詞
        for subject in svos['subject']:
            S = subject[0]
            S_attr = self._Attr_flatten(subject[1])
            for predicate in svos['predicate']:
                P = predicate[0]
                P_attr = []
                for attr in [x for x in predicate[1] if isinstance(x, dict) == False and x != None]:
                    P_attr.append(attr)
                
                # 動詞Attr可以當受詞
                if True in [isinstance(x, dict) for x in predicate[1]]:
                    for attr in [x for x in predicate[1] if isinstance(x, dict) == True]:
                        if 'predicate' in attr.keys() and 'object' in attr.keys():
                            self._results.append([(S, ' '.join(S_attr)), 
                                             (' '.join([P] + [attr['predicate'][0][0]]), ' '.join(P_attr + self._Attr_flatten(attr['predicate'][0][1]))), 
                                             (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                # 動詞Attr不能當受詞
                else:        
                    self._results.append([(S, ' '.join(S_attr)), (P, ' '.join(P_attr))])
    
    def _Complete(self, svos):
        for subject in svos['subject']:
            S = subject[0]
            S_attr = self._Attr_flatten(subject[1])
            for predicate in svos['predicate']:
                P = predicate[0]
                P_attr = self._Attr_flatten(predicate[1])
                for obj in svos['object']:
                    # be動詞 + 受詞是形容詞 + 受詞Attr有dictionary
                    if P in self._be and [x for x in nltk.pos_tag([y for y in obj[0].split(' ') if y != '']) if 'NN' in x[1]] == [] and \
                    True in [isinstance(x, dict) for x in obj[1]]:
                        tmp_P = [P] + [obj[0]]
                        for attr in [x for x in obj[1] if isinstance(x, dict) == False and x != None]:
                            tmp_P.append(attr)
                        for attr in [x for x in obj[1] if isinstance(x, dict) == True]:
                            if 'predicate' in attr.keys() and 'object' in attr.keys():
                                self._results.append([(S, ' '.join(S_attr)), 
                                                     (' '.join(tmp_P + [attr['predicate'][0][0]]), ' '.join(P_attr)), 
                                                     (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                            # attr只有predicate
                            elif 'predicate' in attr.keys():
                                pos = nltk.pos_tag(attr['predicate'][0][0].split(' '))
                                self._results.append([(S, ' '.join(S_attr)), 
                                                     (' '.join(tmp_P + [x[0] for x in pos if 'VB' not in x[1]]), ' '.join(P_attr)), 
                                                     (' '.join([x[0] for x in pos if 'VB' in x[1]]), '')])
                            
                            # 受詞在動詞的Attr中
                            else:
                                for attr in [x for x in predicate[1] if isinstance(x, dict) == True]:
                                    self._results.append([(S, ' '.join(S_attr)), 
                                                         (' '.join(tmp_P + [attr['predicate'][0][0]]), ''), 
                                                         (attr['object'][0][0], ' '.join(self._Attr_flatten(attr['object'][0][1])))])
                    # 正常狀態
                    else:
                        self._results.append([(S, ' '.join(S_attr)), 
                                             (P, ' '.join(P_attr)), 
                                             (obj[0], ' '.join(self._Attr_flatten(obj[1])))])
                        

In [24]:
def Integrate(dataset):
    processed = []
    for sample in tqdm(dataset):
        tmp = {}
        try:
            tmp['id'] = sample['id']
            tmp['date'] = sample['date']
            tmp['title'] = sample['title']
            tmp['title_SVO'] = sample['title_SVO']
            integrate = to_SVO()
            tmp['integrate_SVO'] = integrate.to_SVO(sample['title_SVO'])
            processed.append(tmp)
        except:
            print(sample['id'])
    return processed

In [25]:
n_workers = 4
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(processed) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(processed)
        else:
            batch_end = (len(processed) // n_workers) * (i + 1)

        batch = processed[batch_start: batch_end]
        results[i] = pool.apply_async(Integrate, [batch])

    pool.close()
    pool.join()

end = []
for result in results:
    end += result.get()

100%|██████████| 26623/26623 [00:00<00:00, 27964.61it/s]


In [34]:
# with open('news_preprocessed_integrate.pkl', 'wb') as f:
#     pickle.dump(data, f)

# Event Embedding

In [35]:
with open('news_preprocessed_integrate_1.pkl', 'rb') as f:
    data = pickle.load(f)

In [36]:
len(data)

106483

In [37]:
len(set([x['date'] for x in data]))

2582

### Word Embedding(Bert)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

#### Convert to Token

In [ ]:
# train = [item for sublist in [x['integrate_SVO'] for x in data] for item in sublist]

In [39]:
datas = []
for i in tqdm(list(set([x['date'] for x in data]))):
    tmp = {}
    tmp['date'] = i
    tmp['SVO'] = [item for sublist in [x['integrate_SVO'] for x in data if x['date'] == i] for item in sublist]
    datas.append(tmp)

100%|██████████| 2582/2582 [01:17<00:00, 33.15it/s]


In [40]:
def Tokenize(dataset):
    datas = []
    for data in tqdm(dataset):
        tmp = {}
        tmp['date'] = data['date']
        svos = []
        for i in data['SVO']:
            S = tokenizer.encode(i[0][0], add_special_tokens = False)
            S_attr = tokenizer.encode(i[0][1], add_special_tokens = False)

            if len(i) == 1:
                svos.append([(S, S_attr)])
                continue

            if len(i) >= 2:
                P = tokenizer.encode(i[1][0], add_special_tokens = False)
                P_attr = tokenizer.encode(i[1][1], add_special_tokens = False)

                if len(i) == 2:
                    svos.append([(S, S_attr), (P, P_attr)])
                    continue

            if len(i) == 3:
                O = tokenizer.encode(i[2][0], add_special_tokens = False)
                O_attr = tokenizer.encode(i[2][1], add_special_tokens = False)

                svos.append([(S, S_attr), (P, P_attr), (O, O_attr)])
        tmp['SVO'] = svos
        datas.append(tmp)
    return datas

In [41]:
n_workers = 4
results = [None] * n_workers
with Pool(processes=n_workers) as pool:
    for i in range(n_workers):
        batch_start = (len(datas) // n_workers) * i
        if i == n_workers - 1:
            batch_end = len(datas)
        else:
            batch_end = (len(datas) // n_workers) * (i + 1)

        batch = datas[batch_start: batch_end]
        results[i] = pool.apply_async(Tokenize, [batch])

    pool.close()
    pool.join()

train_token = []
for result in results:
    train_token += result.get()

100%|██████████| 645/645 [00:15<00:00, 42.71it/s]


In [54]:
len(train_token)

2582

In [53]:
# with open('news_token.pkl', 'wb') as f:
#     pickle.dump(train_token, f)

#### Embedding

In [4]:
with open('news_token.pkl', 'rb') as f:
    train_token = pickle.load(f)

In [5]:
len(train_token)

4695

In [6]:
# def Word_Embedding(token, token_attr):
#     with torch.no_grad():
#         if token.shape[0] == 0:
#             return torch.zeros(768)
        
#         a = bert(token.unsqueeze(-1))[0]
#         a = a.view(a.shape[0], a.shape[2])
#         a = a.mean(0)
        
#         # attr空的
#         if token_attr.shape[0] == 0:
#             return a
#         else:
#             attr = bert(token_attr.unsqueeze(-1))[0]
#             attr = attr.view(attr.shape[0], attr.shape[2])
#             attr = attr.mean(0)
#             return (a + attr) / 2

In [6]:
bert.cuda()
train_vector = []
for data in tqdm(train_token):
    tmp = {}
    tmp['date'] = data['date']
    vectors = []
    for t in data['SVO']:
        with torch.no_grad():
            token = torch.tensor(t[0][0]).cuda()
            token_attr = torch.tensor(t[0][1]).cuda()
            if token.shape[0] == 0:
                S = torch.zeros(768).cuda()
            else:
                a = bert(token.unsqueeze(0))[0]
                a = a.mean(1).flatten()

                # attr空的
                if token_attr.shape[0] == 0:
                    S = a
                else:
                    attr = bert(token_attr.unsqueeze(0))[0]
                    attr = attr.mean(1).flatten()
                    S = (a + attr) / 2

            if len(t) == 1:
                vectors.append(torch.stack((S.cpu(), torch.zeros(768), torch.zeros(768))))

            if len(t) >= 2:
                token = torch.tensor(t[1][0]).cuda()
                token_attr = torch.tensor(t[1][1]).cuda()
                if token.shape[0] == 0:
                    P = torch.zeros(768).cuda()
                else:
                    a = bert(token.unsqueeze(0))[0]
                    a = a.mean(1).flatten()

                    # attr空的
                    if token_attr.shape[0] == 0:
                        P = a
                    else:
                        attr = bert(token_attr.unsqueeze(0))[0]
                        attr = attr.mean(1).flatten()
                        P = (a + attr) / 2

                if len(t) == 2:
                    vectors.append(torch.stack((S.cpu(), P.cpu(), torch.zeros(768))))

            if len(t) == 3:
                token = torch.tensor(t[2][0]).cuda()
                token_attr = torch.tensor(t[2][1]).cuda()
                if token.shape[0] == 0:
                    O = torch.zeros(768).cuda()
                else:
                    a = bert(token.unsqueeze(0))[0]
                    a = a.mean(1).flatten()

                    # attr空的
                    if token_attr.shape[0] == 0:
                        O = a
                    else:
                        attr = bert(token_attr.unsqueeze(0))[0]
                        attr = attr.mean(1).flatten()
                        O = (a + attr) / 2
                vectors.append(torch.stack((S.cpu(), P.cpu(), O.cpu())))
    tmp['SVO'] = vectors
    train_vector.append(tmp)

100%|██████████| 4695/4695 [3:07:56<00:00,  2.40s/it]  


In [8]:
len(train_vector)

4695

In [9]:
with open('news_embedding_1.pkl', 'wb') as f:
    pickle.dump(train_vector, f)

### Train AutoEncoder

In [51]:
with open('news_embedding_1.pkl', 'rb') as f:
    train_vector = pickle.load(f)

In [52]:
train_vector = [x for x in train_vector if x['date'] >= '20140101']

In [53]:
train = [item for sublist in [x['SVO'] for x in train_vector] for item in sublist]

In [54]:
len(train)

105859

In [55]:
train_dataloader = DataLoader(train, batch_size=256, shuffle=True)

In [11]:
# class AutoEncoder(nn.Module):
#     def __init__(self):
#         super(AutoEncoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)), # 128, 4, 768
#             nn.Conv2d(128, 32, (3,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
#         )
        
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.ConvTranspose2d(32, 128, (3,1), stride=(1,1), padding=(0,0)), # 128, 4, 768
#             nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
#         )
    
#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
        
#         return encoded, decoded
        

In [56]:
#Best
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.Conv2d(256, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
            nn.Conv2d(128, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.ConvTranspose2d(32, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
            nn.ConvTranspose2d(128, 256, (2,1), stride=(1,1), padding=(0,0)), # 256, 4, 768
            nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded
        

In [9]:
# class AutoEncoder(nn.Module):
#     def __init__(self):
#         super(AutoEncoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Conv2d(1, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 3, 768
#             nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)),
#             nn.Conv2d(128, 32, (2,1), stride=(1,1), padding=(0,0)), # 128, 2, 768
#             nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 32, 1, 768
#         )
        
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.ConvTranspose2d(32, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
#             nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.ConvTranspose2d(256, 1, (3,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
#         )
    
#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
        
#         return encoded, decoded
        

In [57]:
use_gpu = torch.cuda.is_available()
autoencoder = AutoEncoder()
if use_gpu:
    autoencoder.cuda()

In [59]:
criteria = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
EPOCH = 1

for epoch in range(EPOCH):
    cumulate_loss = 0
    for idx, x in tqdm(enumerate(train_dataloader)):
        x = x.unsqueeze(1)
        if use_gpu:
            x = x.cuda()
        latent, reconstruct = autoencoder(x)
        loss = criteria(reconstruct, x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cumulate_loss += loss.item() * x.shape[0]
#         if (idx % 100) == 0:
#             print(loss)
            
    print(f'Epoch { "%03d" % epoch }: Loss : { "%.5f" % (cumulate_loss / len(train))}')

414it [03:46,  1.83it/s]

Epoch 000: Loss : 0.08622


In [60]:
checkpoint_path = 'autoencoder(1channel_2014)_{}.pth'.format(epoch+1) 
torch.save(autoencoder.state_dict(), checkpoint_path)
print('model saved to %s' % checkpoint_path)

model saved to autoencoder(1channel_2014)_1.pth


#### AutoEncoder Validation

# Deep Learning

#### Day-Vector

In [61]:
with open('news_embedding_1.pkl', 'rb') as f:
    data = pickle.load(f)

In [62]:
class Day_vector():
    def __init__(self, input_path, model_path, channel):
        self.channel = channel
        with open(input_path, 'rb') as f:
            self.data = pickle.load(f)
            
        self.autoencoder = AutoEncoder(self.channel)
        self.autoencoder.load_state_dict(torch.load(model_path))
        self.use_gpu = torch.cuda.is_available()
        if self.use_gpu:
            self.autoencoder.cuda()
    
    def DayVector(self, date):
        self._vector = [x['SVO'] for x in self.data if x['date'] == date][0]
        
        if len(self._vector) > 0:
            # Event Embedding
            with torch.no_grad():
                dataloader = DataLoader(self._vector, batch_size=len(self._vector), shuffle=False)
                for x in dataloader:
                    x = x.unsqueeze(1)
                    if self.use_gpu:
                        x = x.cuda()
                    latent, reconstruct = self.autoencoder(x)

                latent = latent.cpu().detach().numpy()
                latent = latent.reshape(len(latent), 768*self.channel)

            return latent.mean(0)
        else:
            return []

class AutoEncoder(nn.Module):
    def __init__(self, channel):
        super(AutoEncoder, self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Conv2d(1, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 3, 768
#             nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)),
#             nn.Conv2d(128, 32, (2,1), stride=(1,1), padding=(0,0)), # 128, 2, 768
#             nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 32, 1, 768
#         )
        
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.ConvTranspose2d(32, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
#             nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.ConvTranspose2d(256, 1, (3,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
#         )
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
            nn.Conv2d(256, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
            nn.Conv2d(128, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.Conv2d(32, 1, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
            nn.ConvTranspose2d(32, 128, (2,1), stride=(1,1), padding=(0,0)), # 128, 3, 768
            nn.ConvTranspose2d(128, 256, (2,1), stride=(1,1), padding=(0,0)), # 256, 4, 768
            nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
        )
        
#         self.encoder = nn.Sequential(
#             nn.Conv2d(1, 256, (2,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.Conv2d(256, 128, (3,1), stride=(1,1), padding=(1,0)), # 128, 4, 768
#             nn.Conv2d(128, 32, (3,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.Conv2d(32, channel, (2,1), stride=(1,1), padding=(0,0)) # 1, 1, 768
#         )
        
#         self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(channel, 32, (2,1), stride=(1,1), padding=(0,0)), # 32, 2, 768
#             nn.ConvTranspose2d(32, 128, (3,1), stride=(1,1), padding=(0,0)), # 128, 4, 768
#             nn.ConvTranspose2d(128, 256, (3,1), stride=(1,1), padding=(1,0)), # 256, 4, 768
#             nn.ConvTranspose2d(256, 1, (2,1), stride=(1,1), padding=(1,0)) # 1, 3, 768
#         )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        return encoded, decoded

In [63]:
day_vector = Day_vector('news_embedding_1.pkl', 'autoencoder(1channel_2014)_1.pth', 1)

In [64]:
date = list(set([x['date'] for x in data if x['date'] >= '20140101']))

In [65]:
datas = {}
for i in tqdm(date):
    day = day_vector.DayVector(i)
    if day != []:
        datas[i] = day
datas = dict([(k,datas[k]) for k in sorted(datas.keys())])

100%|██████████| 2113/2113 [01:23<00:00, 25.30it/s]


In [66]:
len(datas)

2109

In [67]:
with open('Event_embedding(1channel_2014).pkl', 'wb') as f:
    pickle.dump(datas, f)